In [2]:
import numpy as np

In [1]:
import sklearn.ensemble as en

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [13]:
import scipy.stats as st

In [12]:
from sklearn.model_selection import RandomizedSearchCV

In [5]:
import pandas as pd

In [6]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [7]:
labels = pd.read_csv('labels_training_set.csv')

In [8]:
processed = pd.read_csv('features_10.csv')

In [9]:
processed = processed.set_index('person').fillna(0)
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,05-17,05-17,6,1,...,False,False,False,Sao Paulo,Brazil,0.0,0.000000,0.0,0.0,0.0
00091926,viewed product,iPhone 6S,Computer,0,34,372,05-03,05-31,448,28,...,False,False,False,Rio Grande do Sul,Brazil,1679.0,1527.091398,0.0,4649.0,100.0
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,03-26,03-26,10,1,...,False,False,False,Minas Gerais,Brazil,1679.0,1342.333333,0.0,1679.0,849.0
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,05-17,05-26,206,10,...,True,False,False,Sao Paulo,Brazil,369.0,479.366013,379.0,2229.0,250.0
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,05-29,05-29,17,1,...,False,False,False,Minas Gerais,Brazil,2229.0,2229.000000,0.0,2229.0,2229.0


In [10]:
training_features = labels.set_index('person').join(processed)
predict_features = kaggle.set_index('person').join(processed)
training_labels = training_features['label']
training_features = training_features.drop('label',axis=1)

In [11]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,viewed product,iPhone 5s,Smartphone,1,17,23,05-22,05-31,68,9,...,False,False,False,Parana,Brazil,999.0,663.913043,459.0,999.0,269.0
6ec7ee77,visited site,Ninguno,Smartphone,0,1,0,05-28,05-28,2,1,...,False,False,False,Rio Grande do Norte,Brazil,0.0,0.000000,0.0,0.0,0.0
abe7a2fb,viewed product,iPhone 6,Smartphone,0,22,31,02-10,05-29,96,109,...,False,False,True,Sao Paulo,Brazil,1499.0,1184.161290,0.0,1589.0,179.0
34728364,viewed product,Lenovo Vibe K5,Smartphone,0,4,24,05-18,05-27,37,9,...,True,False,False,Sao Paulo,Brazil,479.0,529.916667,0.0,849.0,60.0
87ed62de,viewed product,Samsung Galaxy J7,Computer,0,1,9,05-18,05-19,17,1,...,False,False,False,Santa Catarina,Brazil,549.0,549.000000,0.0,549.0,549.0


## Mean encoding

In [15]:
tg = category_encoders.TargetEncoder()
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [17]:
#train_features_norm = StandardScaler().fit_transform(encoded_training)
#predict_features_norm = StandardScaler().fit_transform(encoded_predict)

In [25]:
params={
    'n_estimators':st.randint(10,50),
    'max_samples':st.uniform(0.1,0.9),
    'max_features':st.uniform(0.1,0.9),
    
}
grid = RandomizedSearchCV(en.BaggingClassifier(oob_score=True,bootstrap=True,bootstrap_features=True),param_distributions=params,scoring='roc_auc',cv=2,verbose=2,n_iter=25)

In [26]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [27]:
grid.fit(xtr,ytr)

Fitting 2 folds for each of 25 candidates, totalling 50 fits
[CV] max_features=0.3853329055816721, max_samples=0.6280243454238846, n_estimators=35 
[CV]  max_features=0.3853329055816721, max_samples=0.6280243454238846, n_estimators=35, total=   2.6s
[CV] max_features=0.3853329055816721, max_samples=0.6280243454238846, n_estimators=35 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s


[CV]  max_features=0.3853329055816721, max_samples=0.6280243454238846, n_estimators=35, total=   2.6s
[CV] max_features=0.37983583078394856, max_samples=0.42446202142042333, n_estimators=48 
[CV]  max_features=0.37983583078394856, max_samples=0.42446202142042333, n_estimators=48, total=   2.9s
[CV] max_features=0.37983583078394856, max_samples=0.42446202142042333, n_estimators=48 
[CV]  max_features=0.37983583078394856, max_samples=0.42446202142042333, n_estimators=48, total=   3.2s
[CV] max_features=0.716293208602481, max_samples=0.31468909686497354, n_estimators=49 
[CV]  max_features=0.716293208602481, max_samples=0.31468909686497354, n_estimators=49, total=   4.8s
[CV] max_features=0.716293208602481, max_samples=0.31468909686497354, n_estimators=49 
[CV]  max_features=0.716293208602481, max_samples=0.31468909686497354, n_estimators=49, total=   4.8s
[CV] max_features=0.5095957100789522, max_samples=0.9872143670545128, n_estimators=32 
[CV]  max_features=0.5095957100789522, max_samp

C:\Users\javif\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:605: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\javif\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:610: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


[CV]  max_features=0.3282389873493578, max_samples=0.6214098790649851, n_estimators=11, total=   0.7s
[CV] max_features=0.3282389873493578, max_samples=0.6214098790649851, n_estimators=11 


...

[CV]  max_features=0.9978513967536067, max_samples=0.8717804242053069, n_estimators=13, total=   3.0s
[CV] max_features=0.9978513967536067, max_samples=0.8717804242053069, n_estimators=13 


C:\Users\javif\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:605: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\javif\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:610: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


[CV]  max_features=0.9978513967536067, max_samples=0.8717804242053069, n_estimators=13, total=   2.8s
[CV] max_features=0.7778233874452978, max_samples=0.579021805871031, n_estimators=14 
[CV]  max_features=0.7778233874452978, max_samples=0.579021805871031, n_estimators=14, total=   2.0s
[CV] max_features=0.7778233874452978, max_samples=0.579021805871031, n_estimators=14 
[CV]  max_features=0.7778233874452978, max_samples=0.579021805871031, n_estimators=14, total=   2.0s
[CV] max_features=0.413530611447662, max_samples=0.17136634941804113, n_estimators=30 
[CV]  max_features=0.413530611447662, max_samples=0.17136634941804113, n_estimators=30, total=   1.4s
[CV] max_features=0.413530611447662, max_samples=0.17136634941804113, n_estimators=30 
[CV]  max_features=0.413530611447662, max_samples=0.17136634941804113, n_estimators=30, total=   1.4s
[CV] max_features=0.751670949051999, max_samples=0.43326894770547175, n_estimators=30 
[CV]  max_features=0.751670949051999, max_samples=0.4332689

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  2.9min finished


RandomizedSearchCV(cv=2, error_score='raise',
          estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=True, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=True, random_state=None,
         verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=25, n_jobs=1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001AB496AB9B0>, 'max_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001AB496ABB00>, 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001AB496ABEB8>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=2)

In [28]:
grid.score(X=xte,y=yte)

0.786138182173916